# Combinación de ambos modelos

En este notebook vamos a recrear el proceso completo con los dos modelos que hemos entrenado, desde cargar una imagen inicial hasta la obtención del archivo MIDI final.

## Cargamos los modelos

In [1]:
from models.extender_midi.model import MelodIAModel, Utilities
from models.foto_a_midi.main import FotoAMIDI

# Cargamos los modelos
MODEL_PARAMS_PATH = "models/pretrained_models/modelo_params_midi.h5"
MODEL_MIDI_EXTENSION_PATH = "models/pretrained_models/modelo_extension_midi.keras"

modelo_params = FotoAMIDI(modelo_ruta=MODEL_PARAMS_PATH, usar_clip=True)
modelo_params.load_modelo()
modelo_extender_midi = MelodIAModel.load_model(MODEL_MIDI_EXTENSION_PATH)

2025-05-18 18:50:04.095632: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2025-05-18 18:50:04.112114: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:467] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1747587004.127568  107744 cuda_dnn.cc:8579] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1747587004.131583  107744 cuda_blas.cc:1407] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
W0000 00:00:1747587004.146364  107744 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking 

Modelo cargado desde: models/pretrained_models/modelo_extension_midi.keras


## Modelo 1: Foto a MIDI

In [2]:
import os
from PIL import Image
from Scripts.python.midi_generator_acordes import ParamsAMIDI

# Cargamos la imagen
IMAGE_PATH = "./models/foto_a_midi/datos/test/noche.jpeg"
im = Image.open(IMAGE_PATH)

In [3]:
# Metemos la imagen en el modelo y predecimos sus parametros
params = modelo_params.predict(im)

I0000 00:00:1747587017.597154  107894 service.cc:152] XLA service 0x7fc36c0e31e0 initialized for platform CUDA (this does not guarantee that XLA will be used). Devices:
I0000 00:00:1747587017.597196  107894 service.cc:160]   StreamExecutor device (0): NVIDIA GeForce RTX 2050, Compute Capability 8.6
2025-05-18 18:50:17.604158: I tensorflow/compiler/mlir/tensorflow/utils/dump_mlir_util.cc:269] disabling MLIR crash reproducer, set env var `MLIR_CRASH_REPRODUCER_DIRECTORY` to enable.
I0000 00:00:1747587017.622681  107894 cuda_dnn.cc:529] Loaded cuDNN version 90501
I0000 00:00:1747587017.797782  107894 device_compiler.h:188] Compiled cluster using XLA!  This line is logged at most once for the lifetime of the process.


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 298ms/step


In [4]:
params

{'tonalidad_value': np.float32(0.0),
 'tempo': np.float32(20.0),
 'duracion_media': np.float32(0.59843194),
 'sigma': np.float32(0.55461925),
 'velocidad_media': np.float32(0.0),
 'densidad_media': np.float32(0.0),
 'caracter_melodico': np.float32(0.0),
 'usar_acordes': np.float32(0.0),
 'proporcion_acordes': np.float32(0.0),
 'rango_octavas': np.float32(1.0)}

In [5]:
from models.foto_a_midi.modelo import clip_parametros

# Convertimos a los tipos correctos
keys = params.keys()
values = clip_parametros(list(params.values()))
params2 = dict(zip(keys, values))
print(params2)

{'tonalidad_value': np.float32(0.0), 'tempo': 20, 'duracion_media': np.float32(0.59843194), 'sigma': np.float32(0.55461925), 'velocidad_media': np.float32(0.0), 'densidad_media': np.float32(0.0), 'caracter_melodico': np.float32(0.0), 'usar_acordes': 0, 'proporcion_acordes': np.float32(0.0), 'rango_octavas': 1}


In [11]:
MIDI_PATH = "prueba.mid"

# Generamos el midi a partir de los parámetros obtenidos
midi_basico = ParamsAMIDI(**params2)
midi_basico.generar_midi(MIDI_PATH)

# Si el midi tiene menos de 25 notas, lo concatenamos
import pretty_midi

pm = pretty_midi.PrettyMIDI(MIDI_PATH)

Archivo MIDI generado: prueba.mid


In [12]:
pm.instruments

[]

In [29]:
num_notas = len(pm.instruments[0].notes)

if (num_notas < 25):
    n = 25 / num_notas + 1
    aux = pm
    for i in range(0, n):
        aux = Utilities.concatenate_pretty_midi(pm, aux)
    pm = aux

IndexError: list index out of range

## Modelo 2: Extensión del archivo MIDI generado

In [ ]:
import pretty_midi

df = Utilities.midi_path_to_df(MIDI_PATH)
notes = Utilities.midi_to_numpy(pm)
notes_sequence = notes[:25] # primeras 25 notas
notes = MelodIAModel.normalize(notes_sequence) 

Generamos las nuevas notas con el modelo

In [ ]:
# Se pueden cambiar el número de predicciones y la temperatura
generated_notes = MelodIAModel.predict_sequence(modelo_extender_midi, notes, num_predictions=25, temperature=0.8)

generated_notes = MelodIAModel.desnormalize(generated_notes)
generado = Utilities.notes_to_midi(generated_notes)

Unimos el fragmento MIDI original con el generado

In [ ]:
original = Utilities.notes_to_midi(notes_sequence)
combined = Utilities.concatenate_pretty_midi(original, generado)

Lo visualizamos

In [ ]:
Utilities.plot_piano_roll(combined, 40, 100)

También se puede escuchar

In [ ]:
Utilities.display_audio(combined)

Por último lo guardamos

In [ ]:
combined.write("midi_final.mid")